In [1]:
import numpy as np, pandas as pd, seaborn as sns, matplotlib.pyplot as plt

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
df = pd.read_csv('http://www.ppgia.pucpr.br/~jean.barddal/datascience/enron.csv')

In [3]:
df.head(5)

,bonus,deferral_payments,deferred_income,director_fees,email_address,exercised_stock_options,expenses,from_messages,from_poi_to_this_person,from_this_person_to_poi,loan_advances,long_term_incentive,other,poi,restricted_stock,restricted_stock_deferred,salary,shared_receipt_with_poi,to_messages,total_payments,total_stock_value,name
0,4175000.0,2869717.0,-3081055.0,NaN,phillip.allen@enron.com,1729541.0,13868.0,2195.0,47.0,65.0,NaN,304805.0,152.0,False,126027.0,-126027.0,201955.0,1407.0,2902.0,4484442.0,1729541.0,ALLEN PHILLIP K
1,NaN,178980.0,NaN,NaN,NaN,257817.0,3486.0,NaN,NaN,NaN,NaN,NaN,NaN,False,NaN,NaN,NaN,NaN,NaN,182466.0,257817.0,BADUM JAMES P
2,NaN,NaN,-5104.0,NaN,james.bannantine@enron.com,4046157.0,56301.0,29.0,39.0,0.0,NaN,NaN,864523.0,False,1757552.0,-560222.0,477.0,465.0,566.0,916197.0,5243487.0,BANNANTINE JAMES M
3,1200000.0,1295738.0,-1386055.0,NaN,NaN,6680544.0,11200.0,NaN,NaN,NaN,NaN,1586055.0,2660303.0,False,3942714.0,NaN,267102.0,NaN,NaN,5634343.0,10623258.0,BAXTER JOHN C
4,400000.0,260455.0,-201641.0,NaN,frank.bay@enron.com,NaN,129142.0,NaN,NaN,NaN,NaN,NaN,69.0,False,145796.0,-82782.0,239671.0,NaN,NaN,827696.0,63014.0,BAY FRANKLIN R


In [4]:
# Método de Tukey:
# Passo 1: Obter as colunas numéricas.
# Passo 2: Criar uma lista para os outliers identificados durante o processo.
# Passo 3: Um laço de repetição, incrementando a lista acima com os outliers de cada variável.
# Passo 4: Analisar os outliers mais frequentes.

outliers_global = [] # todos os índices marcados como outliers.

# Laço de repetição por coluna.
for col in df.select_dtypes(include='number').columns.values:
    # Calculando o método de Tukey para a coluna col.

    # Calculando os quartis(Q3 e Q1).
    q1 = df[col].quantile(0.25)
    q3 = df[col].quantile(0.75)
    
    # Calculando o intervalo inter-quartis:
    iqr = q3 - q1

    # Calculando limites superiores e inferiores.
    sup = q3 + 1.5*iqr
    inf = q1 - 1.5*iqr

    # Selecionando valores externos ao intervalo (i.e., > sup ou < inf)
    ix_outliers_col = list(df[(df[col] > sup) | (df[col] < inf)].index) 

    # Adicionando os índices desta coluna à lista global.
    outliers_global.extend(ix_outliers_col)

#print(outliers_global)

from collections import Counter
for ix, contagem in Counter(outliers_global).most_common(10):
    print(f'{ix} \t {contagem} \t {df.loc[ix]["name"]}')

130 	 13 	 TOTAL
47 	 9 	 FREVERT MARK A
7 	 8 	 BELDEN TIMOTHY N
79 	 8 	 LAY KENNETH L
78 	 7 	 LAVORATO JOHN J
139 	 7 	 WHALLEY LAWRENCE G
122 	 6 	 SKILLING JEFFREY K
0 	 5 	 ALLEN PHILLIP K
75 	 5 	 KITCHEN LOUISE
3 	 5 	 BAXTER JOHN C


In [5]:
# Isolation Forest
from sklearn.ensemble import IsolationForest

# Imputando base de dados com a média nas colunas númericas.
df_num = df.select_dtypes(include='number')
df_num.fillna(df_num.mean(), inplace=True)

# Criando o modelo de isolation forest.
isof = IsolationForest(random_state=101)
isof.fit(df_num)

/usr/local/lib/python3.6/dist-packages/pandas/core/generic.py:6245: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)


IsolationForest(behaviour='deprecated', bootstrap=False, contamination='auto',
                max_features=1.0, max_samples='auto', n_estimators=100,
                n_jobs=None, random_state=101, verbose=0, warm_start=False)

In [6]:
# Predição (outlier ou inlier).
df['inlier'] = (isof.predict(df_num))

In [7]:
df[df['inlier'] == -1]

,bonus,deferral_payments,deferred_income,director_fees,email_address,exercised_stock_options,expenses,from_messages,from_poi_to_this_person,from_this_person_to_poi,loan_advances,long_term_incentive,other,poi,restricted_stock,restricted_stock_deferred,salary,shared_receipt_with_poi,to_messages,total_payments,total_stock_value,name,inlier
47,2000000.0,6426990.0,-3367011.0,NaN,mark.frevert@enron.com,10433518.0,86987.0,21.0,242.0,6.0,2000000.0,1617011.0,7427621.0,False,4188667.0,NaN,1060932.0,2979.0,3275.0,17252530.0,14622185.0,FREVERT MARK A,-1
73,1000000.0,NaN,NaN,NaN,steven.kean@enron.com,2022048.0,41953.0,6759.0,140.0,387.0,NaN,300000.0,1231.0,False,4131594.0,NaN,404338.0,3639.0,12754.0,1747522.0,6153642.0,KEAN STEVEN J,-1
78,8000000.0,NaN,NaN,NaN,john.lavorato@enron.com,4158995.0,49537.0,2585.0,528.0,411.0,NaN,2035380.0,1552.0,False,1008149.0,NaN,339288.0,3962.0,7259.0,10425757.0,5167144.0,LAVORATO JOHN J,-1
79,7000000.0,202911.0,-300000.0,NaN,kenneth.lay@enron.com,34348384.0,99832.0,36.0,123.0,16.0,81525000.0,3600000.0,10359729.0,True,14761694.0,NaN,1072321.0,2411.0,4273.0,103559793.0,49110078.0,LAY KENNETH L,-1
130,97343619.0,32083396.0,-27992891.0,1398517.0,NaN,311764000.0,5235198.0,NaN,NaN,NaN,83925000.0,48521928.0,42667589.0,False,130322299.0,-7576788.0,26704229.0,NaN,NaN,309886585.0,434509511.0,TOTAL,-1


In [8]:
isof.score_samples(df_num)

array([-0.43796161, -0.33752391, -0.3822066 , -0.37448888, -0.35932113,
       -0.34931212, -0.4339837 , -0.48605681, -0.40752107, -0.32174731,
       -0.36022419, -0.47012818, -0.32751419, -0.3295305 , -0.33853019,
       -0.34868135, -0.32186257, -0.36047016, -0.33962188, -0.38364367,
       -0.36009179, -0.37785522, -0.33956421, -0.35650669, -0.31655597,
       -0.31987736, -0.35721654, -0.32793128, -0.3488137 , -0.33454265,
       -0.3387507 , -0.42746017, -0.35864048, -0.34441318, -0.36729849,
       -0.32907346, -0.35075494, -0.33618396, -0.34274221, -0.33275913,
       -0.33609615, -0.3391858 , -0.3391409 , -0.32142655, -0.33586259,
       -0.33076229, -0.34942872, -0.59486586, -0.32720384, -0.34812493,
       -0.34253061, -0.31419643, -0.36216724, -0.31899455, -0.33430111,
       -0.32607904, -0.32637488, -0.36991326, -0.38523062, -0.36338038,
       -0.33500652, -0.34805542, -0.33446764, -0.32957896, -0.3374389 ,
       -0.45417162, -0.34616079, -0.32065002, -0.37882574, -0.36

In [15]:
 # todos os índices marcados como outliers.
from collections import Counter

def by_tukey(most_common=10):
    outliers_global = []
    for col in df.select_dtypes(include='number').columns.values:
        q1, q3 = df[col].quantile(0.25), df[col].quantile(0.75)
        iqr = q3 - q1
        outliers_global.extend(list(df[(df[col] > (q3 + 1.5*iqr)) | (df[col] < (q1 - 1.5*iqr))].index))
    return [df.loc[ix]["name"] for ix, cont in Counter(outliers_global).most_common(most_common)]
  
def by_isof(n='isof_match'):
    from sklearn.ensemble import IsolationForest
    df_num = df.select_dtypes(include='number')
    df_num.fillna(df_num.mean(), inplace=True)
    isof = IsolationForest(random_state=101)
    isof.fit(df_num)
    df[n] = (isof.predict(df_num))

In [29]:
result = [df[df['name']==str(i)]['poi'].tolist()[0] for i in by_tukey()]
r = len(list(filter(lambda x: x == True, result)))/ len(result)
print("fraudsters: ",r)

fraudsters:  0.3


In [36]:
#by_isof()
result = df[df['inlier']==-1]['poi'].tolist()
r = len(list(filter(lambda x: x == True, result))) / len(result)
print("Fraudsters: ", r)

Fraudsters:  0.3333333333333333
